<h2>

Program ini bertujuan untuk merapihkan format yang digunakan

</h2>

# **Library**

In [1]:
import pandas as pd
import os, time, csv, re
import matplotlib.pyplot as plt
import pycountry

# **Parameter**

In [22]:
review_keywords = ['review', 'survey', 'overview', 'meta-analysis', 'systematic review', 
                   'state of the art', 'literature review', 'comparative analysis', 
                   'synthesis', 'bibliometric analysis']
                   
PATH = './result/'.replace('/', '\\')

uniform_article_types = {
    'Review Paper': 'Review Paper',
    'Review': 'Review Paper',
    'Conference Review': 'Review Paper',
    'Article': 'Research Article',
    'article': 'Research Article',
    'Journal Article': 'Research Article',
    'Conference Paper': 'Conference Paper',
    'conference':'Conference Paper',
    'Book Chapter': 'Book Chapter',
    'Research Article': 'Research Article',
    'research-article': 'Research Article',
    'review-article': 'Review Paper',
    'editorial': 'Editorial',
    'poster': 'Poster',
    'short-paper': 'Research Article',
    'survey': 'Survey Paper',
    'ORIGINAL ARTICLE': 'Research Article',
    'ORIGINAL PAPER': 'Research Article',
    'ORIGINAL RESEARCH': 'Research Article',
    'ORIGINAL RESEARCH PAPER': 'Research Article',
    'RESEARCH ARTICLE': 'Research Article',
    'REVIEW': 'Review Paper',
    'Research Articles': 'Research Article',
    'Review Article': 'Review Paper',
    'SPECIAL ISSUE ARTICLE': 'Special Issue Article',
    'SPECIAL ISSUE PAPER': 'Special Issue Paper',
    'SURVEY ARTICLE': 'Survey Paper',
    'none': 'None',
    'Volume: 4'                  :'Research Article',
    'Volume: 7'                  :'Research Article',
    'Volume: 9'                  :'Research Article',
    'Volume: 10'                 :'Research Article',
    'Volume: 11'                 :'Research Article',
    'Volume: 12'                 :'Research Article',
    'Volume: 10, Issue: 2'       :'Research Article',
    'Volume: 11, Issue: 1'       :'Research Article',
    'Volume: 11, Issue: 6'       :'Research Article',
    'Volume: 16, Issue: 8'       :'Research Article',
    'Volume: 18, Issue: 4'       :'Research Article',
    'Volume: 20, Issue: 2'       :'Research Article',
    'Volume: 20, Issue: 6'       :'Research Article',
    'Volume: 21, Issue: 2'       :'Research Article',
    'Volume: 21, Issue: 5'       :'Research Article',
    'Volume: 23, Issue: 4'       :'Research Article',
    'Volume: 24, Issue: 5'       :'Research Article',
    'Volume: 24, Issue: 7'       :'Research Article',
    'Volume: 70, Issue: 3'       :'Research Article',
    'Volume: 72, Issue: 7'       :'Research Article',
    'Volume: 10, Issue: 13'      :'Research Article',
    'Volume: 19, Issue: 18'      :'Research Article',
    'Volume: 19, Issue: 22'      :'Research Article',
    'Volume: 22, Issue: 10'      :'Research Article',
    'Volume: 22, Issue: 13'      :'Research Article',
    'Volume: 23, Issue: 15'      :'Research Article',
    'Volume: 24, Issue: 10'      :'Research Article',
    'Volume: 24, Issue: 15'      :'Research Article',
    'Volume: 37, Issue: 11'      :'Research Article',
    'Volume: 7, Issue: 10'       :'Research Article',
    'Volume: 8, Issue: 10'       :'Research Article',
    'Volume: 8, Issue: 14'       :'Research Article',
    'Volume: 9, Issue: 19'       :'Research Article',
    'Volume: 8, Issue: 4'        :'Research Article',
    'Volume: 9, Issue: 5'        :'Research Article',
    'Volume: 9, Issue: 6'        :'Research Article'
}


properties = [
    'supervised',
    'unsupervised',
    'semi-supervised',
    'reinforcement learning',
    'transfer learning',
    'self-supervised learning',
    'multi-task learning',
    'active learning',
    'graph neural networks',
    'convolutional neural networks',
    'recurrent neural networks',
    'physics-informed neural networks (PINNs)',
    'physics-constrained learning',
    'neural operator learning',
    'variational methods',
    'data-driven methods',
    'model-based learning',
    'hybrid methods',
    'stochastic learning',
    'probabilistic methods',
    'meta-learning',
    'optimization-based approaches',
    'adaptive learning',
    'deep learning',
    'machine learning',
    'inverse modeling',
    'domain decomposition methods',
    'multiscale modeling',
    'transferable representations'
]

property_groups = [
    'acoustic wave',
    'seismic wave',
    'electromagnetic wave',
    'elastic wave',
    'ocean wave',
    'shock waves',
    'quantum wave',
    'plasma waves',
    'gravity waves',
    'light waves',
    'surface waves',
    'body waves',
    'fluid dynamics',
    'nonlinear wave',
    'hyperbolic wave equations',
    'wave propagation in anisotropic media',
    'wave scattering',
    'wave dispersion',
    'wave interference',
    'standing waves',
    'traveling waves',
    'wave equations in bounded domains',
    'wave equations in unbounded domains',
    'multi-dimensional wave equations',
    'stochastic wave equations',
    'inverse wave problems',
    'adaptive wave modeling'
]

# Replace the article_type column with uniform types using the mapping

def joinfile(publisher, show = False):
    FILE_PATH = PATH + publisher
    dfs = []
    for file in os.listdir(FILE_PATH):
        file = os.path.join(FILE_PATH, file)
        if file.endswith('.csv'):
            dfs.append(
                pd.read_csv(file)
            )
            if show:
                print(file)
        elif file.endswith('.xlsx'):
            dfs.append(
                pd.read_excel(file)
            )

            if show:
                print(file)
    df = pd.concat(dfs, ignore_index=True)
    print(f"Number of article in {publisher} : {len(df.index)}")
    return df

def filterDataframe(df):
    for col in df.columns:
        df[col] = df[col].fillna('none')
    df = df.reset_index(drop=True)
    return df

def filterNumberOfAffiliation(affiliations_string):
    # Use regex to find all affiliations
    affiliations_list = re.findall(r'\((.*?)\)', affiliations_string)
    return len(affiliations_list)

def filterCountries(country_string):
    occurrences = country_string.split(',')
    found_countries = []

    for country in pycountry.countries:
        for occurrence in occurrences:
            if country.name in occurrence:
                found_countries.append(country.name)

    return ','.join(found_countries)

def is_review_paper(row):
    if 'conference' not in row['article_type'].lower():
        text = f"{row['title']} {row['keyword']} {row['abstract']}".lower()
        for word in review_keywords:
            if word in text:
                return 'Review Paper'
    return row['article_type']

def count_unique_countries(country_string):
    if not country_string.strip():  # Handle empty strings
        return 0
    countries = country_string.split(',')
    unique_countries = set(countries)
    return len(unique_countries)

# Apply the function to the DataFrame and store in a new column
def count_properties(row):
    text = f"{row['title']} {row['abstract']} {row['keyword']}".lower()
    return sum(1 for prop in properties if prop.lower() in text)

def count_properties(row):
    text = f"{row['title']} {row['abstract']} {row['keyword']}".lower()
    return sum(1 for prop in property_groups if prop.lower() in text)

# Apply the function to calculate the new column


def filterArticleType(df):
    df['article_type'] = df.apply(is_review_paper, axis=1)
    df['article_type'] = df['article_type'].replace(uniform_article_types)
    df['number_of_citation'] = df['number_of_citation'].replace({'none':0})
    df['number_of_citation'] = pd.to_numeric(df['number_of_citation'], errors='coerce')
    df['title'] = df['title'].astype('string') 
    df['article_type'] = df['article_type'].astype('string') 
    df['publisher'] = df['publisher'].astype('string') 
    df['keyword'] = df['keyword'].astype('string') 
    df['abstract'] = df['abstract'].astype('string') 
    df['publication_title'] = df['publication_title'].astype('string') 
    df['authors'] = df['authors'].astype('string') 
    df['affiliations'] = df['affiliations'].astype('string') 
    df['countries'] = df['countries'].astype('string') 
    df['countries'] = df['countries'].apply(filterCountries)
    df['number_of_countries'] = df['countries'].apply(count_unique_countries)
    df['number_of_countries'] = pd.to_numeric(df['number_of_countries'], errors='coerce')
    
    df['number_of_found_property_group'] = df.apply(count_properties, axis=1)
    df['number_of_found_property_group'] = pd.to_numeric(df['number_of_found_property_group'], errors='coerce')

    df['number_of_found_properties'] = df.apply(count_properties, axis=1)
    df['number_of_found_properties'] = pd.to_numeric(df['number_of_found_properties'], errors='coerce')
    df['number_of_affiliations'] = df['affiliations'].apply(filterNumberOfAffiliation)
    df = df[df['countries'] != '']
    df = df.reset_index(drop=True)
    
    return df

def filterDatetime(df):
    try:
        df['publish_date'] = pd.to_datetime(df['publish_date'], format='%d %B %Y', errors='coerce')
    except ValueError:
        df['publish_date'] = pd.to_datetime(df['publish_date'], format='%Y-%m-%d', errors='coerce')

    return df

In [21]:
print(df)

                                                 title  \
0    Modeling Shear Wave Propagation in an Incompre...   
1    Numerical Solver-Independent Seismic Wave Simu...   
2    Physics-informed neural networks for modeling ...   
3    Modeling Shear Wave Propagation in an Incompre...   
4    Modeling Shear Wave Propagation in an Incompre...   
..                                                 ...   
148  Seismic Inversion Based on Acoustic Wave Equat...   
149  Physics-Informed Neural Networks for Solving U...   
150  Seismic Inversion Based on Acoustic Wave Equat...   
151  Physics-Informed Neural Networks for Solving U...   
152  Physics-Informed Neural Networks for Solving U...   

                                               link  number_of_citation  \
0     https://ieeexplore.ieee.org/document/9958579/                   2   
1    https://ieeexplore.ieee.org/document/10229173/                   3   
2    https://ieeexplore.ieee.org/document/10308026/                   0   
3  

# **Springer**

In [3]:
df = joinfile('Springer')
p1 = len(df.index)
print(f"\nData {publisher}: (p1: {p1})")

df = filterDataframe(df)
df = filterArticleType(df)


p2 = len(df.index)
print(f"Data {publisher}: (p2: {p2}) | diff : {p1-p2}\n")

for i in range(len(df.index)):
    temp = df.loc[i, 'publish_date']
    temp_datetime = pd.to_datetime(temp, format='%d %B %Y', errors='coerce')

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='%Y', errors='coerce')
        #print(temp_datetime)

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='%B %Y', errors='coerce')

    if pd.isnull(temp_datetime):
        print('success')
    else:
        df.loc[i, 'publish_date'] = temp

p3 = len(df.index)
print(f"Data {publisher}: (p3: {p3}) | diff : {p2-p3}\n")

df_springer = df

Number of article in Springer : 1116


NameError: name 'publisher' is not defined

# **Elsevier**

In [ ]:
df = joinfile('Elsevier', True)
p1 = len(df.index)
print(f"\nData {publisher}: (p1: {p1})")

df = filterDataframe(df)
df = filterArticleType(df)


p2 = len(df.index)
print(f"Data {publisher}: (p2: {p2}) | diff : {p1-p2}\n")

for i in range(len(df.index)):
    temp = df.loc[i, 'publish_date']
    temp_datetime = pd.to_datetime(temp, format='%d %B %Y', errors='coerce')

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='%Y', errors='coerce')
        #print(temp_datetime)

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='%B %Y', errors='coerce')

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='%Y-%m-%d', errors='coerce')

    if pd.isnull(temp_datetime):
        print('success')
    else:
        df.loc[i, 'publish_date'] = temp

p3 = len(df.index)
print(f"Data {publisher}: (p3: {p3}) | diff : {p2-p3}\n")

df_elsevier = df

.\results\Elsevier\Sciencedirect_233_articles_18450.csv
.\results\Elsevier\scopus_314_articles.xlsx
Number of article in Elsevier : 564

Data Wiley: (p1: 564)
Data Wiley: (p2: 492) | diff : 72

Data Wiley: (p3: 492) | diff : 0



In [ ]:
df

,title,link,number_of_citation,article_type,publisher,keyword,abstract,publish_date,publication_title,authors,affiliations,number_of_affiliations,countries,number_of_countries,number_of_found_property_group,number_of_found_properties
0,Intelligent data routing strategy based on fed...,https://doi.org/10.1016/j.measen.2023.101012,3,None,Sciencedirect,"Data routing,Federated deep reinforcement lear...",The high-speed data routing in the Internet of...,12 January 2024,Measurement: Sensors,"Suresh S. Sebastin,Prabhu V.,Parthasarathy V.,...","(Anna University, Chennai, Tamil Nadu, India),...",5,"India,India,India,India,India",5,0,0
1,WOAD3QN-RP: An intelligent routing protocol in...,https://doi.org/10.1016/j.eswa.2023.123089,3,None,Sciencedirect,"Wireless sensor networks,Clustering,Routing,Sw...",Wireless Sensor Networks (WSN) are a crucial p...,12 January 2024,Expert Systems with Applications,"Yang Xuan,Yan Jiaqi,Wang Desheng,Xu Yonggang,H...",(School of Information and Control Engineering...,5,"China,China,China,China,China,India,India,Indi...",5,7,0
2,A reinforcement learning based routing protoco...,https://doi.org/10.1016/j.future.2023.08.006,8,None,Sciencedirect,"Reinforcement learning,Routing,Software define...",Critical event reporting Wireless Sensor Netwo...,24 August 2023,Future Generation Computer Systems,"Moussa Noureddine,Nurellari Edmond,Azbeg Kebir...","(LabSIV, Department of Computer Science, Facul...",8,"Morocco,Morocco,Morocco,Morocco,Morocco,Morocc...",8,1,4
3,CRLM: A cooperative model based on reinforceme...,https://doi.org/10.1016/j.comnet.2023.110019,3,None,Sciencedirect,"Wireless sensor networks,Hierarchical routing,...","In wireless sensor networks, reasonable cluste...",19 September 2023,Computer Networks,"Wang Zhendong,Shao Liwei,Yang Shuxin,Wang Junl...","(School of Information Engineering, Jiangxi Un...",5,"China,China,China,China,China",5,3,0
4,3R: A reliable multi agent reinforcement learn...,https://doi.org/10.1016/j.comnet.2023.110073,1,None,Sciencedirect,"Routing,Reinforcement learning,Trust managemen...",Interest in the Wireless Medical Sensor Networ...,27 October 2023,Computer Networks,"Hajar Muhammad Shadi,Kalutarage Harsha Kumara,...","(School of Computing, Robert Gordon University...",3,"China,China,China",3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,A mobile agents-based real-time mechanism for ...,https://api.elsevier.com/content/abstract/scop...,12,Conference Paper,Scopus,Internet | Mobile agent | Wireless sensor network,Due to the variety of applications and their i...,2006-12-01,Proceedings of IEEE ICIA 2006 - 2006 IEEE Inte...,"Bai Jieyin,Zang Chuanzhi,Wang Tianran,Yu Haibin",(Shenyang Institute of Automation Chinese Acad...,4,"China,China,China,China",4,0,0
488,Sensor networks routing via Bayesian exploration,https://api.elsevier.com/content/abstract/scop...,7,Conference Paper,Scopus,none,There is increasing research interest in solvi...,2006-12-01,Proceedings - Conference on Local Computer Net...,"Hao Shuang,Wang Ting","(The University of British Columbia,Vancouver,...",2,"Canada,Canada",2,0,0
489,Fuzzy reinforcement learning for routing in wi...,https://api.elsevier.com/content/abstract/scop...,3,Book Chapter,Scopus,Multiagent systems | Q-learning | Reinforcemen...,The wireless sensor networks (WSNs) are compos...,2006-12-01,"Computational Intelligence, Theory and Applica...",Martyna Jerzy,"(Uniwersytet Jagielloński,Krakow,Poland)",1,Poland,1,1,0
490,Constrained flooding: A robust and efficient r...,https://api.elsevier.com/content/abstract/scop...,21,Conference Paper,Scopus,none,Flooding protocols for wireless networks in ge...,2006-11-22,Proceedings - International Conference on Adva...,"Zhang Ying,Fromherz Markus","(Palo Alto Research Center Incorporated,Palo A...",2,"United States,United States",2,0,0


# **IEEE**

In [11]:
df = joinfile('IEEE', False)
p1 = len(df.index)
#print(f"\nData {IEEE}: (p1: {p1})")

df = filterDataframe(df)
df = filterArticleType(df)


p2 = len(df.index)
print(f"Data {publisher}: (p2: {p2}) | diff : {p1-p2}\n")

for i in range(len(df.index)):
    temp = df.loc[i, 'publish_date']
    temp_datetime = pd.to_datetime(temp, format=' %d %B %Y ', errors='coerce')
    

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='Date of Publication: %d %B %Y ', errors='coerce')
        #print(temp_datetime)

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='Date of Publication: %B %Y ', errors='coerce')

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='%Y-%m-%d', errors='coerce')

    if pd.isnull(temp_datetime):
        print('success')
    else:
        df.loc[i, 'publish_date'] = temp

p3 = len(df.index)
print(f"Data {publisher}: (p3: {p3}) | diff : {p2-p3}\n")

df_IEEE = df

Number of article in IEEE : 153


KeyError: 'number_of_countries'

# **ACM**

In [ ]:
df = joinfile('ACM', False)
p1 = len(df.index)
print(f"\nData: (p1: {p1})")

df = filterDataframe(df)
df = filterArticleType(df)


p2 = len(df.index)
print(f"Data: (p2: {p2}) | diff : {p1-p2}")

for i in range(len(df.index)):
    temp = df.loc[i, 'publish_date']
    temp_datetime = pd.to_datetime(temp, format='%d %B %Y', errors='coerce')

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='%Y', errors='coerce')
        #print(temp_datetime)

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='%B %Y', errors='coerce')

    if pd.isnull(temp_datetime):
        print('success')
    else:
        df.loc[i, 'publish_date'] = temp

p3 = len(df.index)
print(f"Data: (p3: {p3}) | diff : {p2-p3}\n")


df_ACM = df

Number of article in ACM : 1334

Data: (p1: 1334)
Data: (p2: 1053) | diff : 281
Data: (p3: 1053) | diff : 0



# **Wiley**

In [ ]:
df = joinfile('Wiley', False)
p1 = len(df.index)
print(f"\nData: (p1: {p1})")

df = filterDataframe(df)
df = filterArticleType(df)


p2 = len(df.index)
print(f"Data: (p2: {p2}) | diff : {p1-p2}")

for i in range(len(df.index)):
    temp = df.loc[i, 'publish_date']
    temp_datetime = pd.to_datetime(temp, format='%d %B %Y', errors='coerce')

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='%Y', errors='coerce')
        #print(temp_datetime)

    if pd.isnull(temp_datetime):
        temp_datetime = pd.to_datetime(temp, format='%B %Y', errors='coerce')

    if pd.isnull(temp_datetime):
        print('success')
    else:
        df.loc[i, 'publish_date'] = temp

p3 = len(df.index)
print(f"Data: (p3: {p3}) | diff : {p2-p3}\n")
df_wiley = df

Number of article in Wiley : 474

Data: (p1: 474)
Data: (p2: 430) | diff : 44
Data: (p3: 430) | diff : 0



In [ ]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# **Merge Dataset**

In [23]:
def process_publish_date(df):
    for i in range(len(df.index)):
        temp = df.loc[i, 'publish_date']
        temp_datetime = pd.to_datetime(temp, format='%d %B %Y', errors='coerce')  # Try 'day Month year'

        if pd.isnull(temp_datetime):
            temp_datetime = pd.to_datetime(temp, format='%Y', errors='coerce')    # Try 'year'

        if pd.isnull(temp_datetime):
            temp_datetime = pd.to_datetime(temp, format='%B %Y', errors='coerce')  # Try 'Month year'

        if pd.isnull(temp_datetime):
            temp_datetime = pd.to_datetime(temp, format='%Y-%m-%d', errors='coerce')  # Try 'YYYY-MM-DD'

        if pd.isnull(temp_datetime):
            temp_datetime = pd.to_datetime(temp, format='Date of Publication: %d %B %Y', errors='coerce')  # Try 'Date of Publication: day Month year'

        if pd.isnull(temp_datetime):
            temp_datetime = pd.to_datetime(temp, format='Date of Publication: %B %Y', errors='coerce')  # Try 'Date of Publication: Month year'

        # If successfully converted, update the dataframe
        if not pd.isnull(temp_datetime):
            df.loc[i, 'publish_date'] = temp_datetime

    return df

df_ieee     = joinfile('IEEE' )
# df_springer = joinfile('Springer')
# df_elsevier = joinfile('Elsevier')
# df_acm      = joinfile('ACM'  )
# df_wiley    = joinfile('Wiley')
# df = pd.concat([df_springer, df_elsevier, df_ieee, df_acm, df_wiley], ignore_index=True)
df = pd.concat([df_ieee], ignore_index=True)
p1 = len(df.index)
df_original = df.copy()

df = filterDataframe(df)
df = filterArticleType(df)
df = process_publish_date(df)
df_2 = df_original[~df_original.index.isin(df.index)].reset_index(drop=True)
p2 = len(df.index)
print()

df_before_dedup = df.copy() 
df = df.drop_duplicates(subset='title')
df_3 = df_before_dedup[~df_before_dedup.index.isin(df.index)].reset_index(drop=True)  # Rows that are duplicates
p3 = len(df.index)


print(f"\n\nFrom: {p1} | Successfully retrieved: {p3} | Duplicated: {p2-p3} | Broken: {p1-p2}\n")

df['publish_date'] = pd.to_datetime(df['publish_date'], errors='coerce')
df.to_excel('Merged_all_data.xlsx', index=False)
df_2.to_excel('incomplete_articles.xlsx', index=False)
df_3.to_excel('duplicate_groups.xlsx', index=False)

Number of article in IEEE : 153



From: 153 | Successfully retrieved: 15 | Duplicated: 76 | Broken: 62



C:\Users\user\AppData\Local\Temp\ipykernel_3268\3361664994.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['publish_date'] = pd.to_datetime(df['publish_date'], errors='coerce')


In [24]:
df_3

,title,link,number_of_citation,article_type,publisher,keyword,abstract,publish_date,publication_title,authors,affiliations,countries,number_of_countries,number_of_found_property_group,number_of_found_properties,number_of_affiliations
0,Numerical Solver-Independent Seismic Wave Simu...,https://ieeexplore.ieee.org/document/10229173/,3,Research Article,IEEE,"Acoustic wave equation modeling,numerical solv...",Solving the wave equation is an essential step...,Date of Publication: 24 August 2023,IEEE Geoscience and Remote Sensing Letters,"Jingbo Zou,Cai Liu,Chao Song,Pengfei Zhao,Jing...","(Department of Geophysics, College of Geo-Expl...","China,China,China,China,China,China,China,China",1,2,2,8
1,Numerical Solver-Independent Seismic Wave Simu...,https://ieeexplore.ieee.org/document/10229173/,3,Research Article,IEEE,"Acoustic wave equation modeling,numerical solv...",Solving the wave equation is an essential step...,Date of Publication: 24 August 2023,IEEE Geoscience and Remote Sensing Letters,"Jingbo Zou,Cai Liu,Chao Song,Pengfei Zhao,Jing...","(Department of Geophysics, College of Geo-Expl...","China,China,China,China,China,China,China,China",1,2,2,8
2,Numerical Solver-Independent Seismic Wave Simu...,https://ieeexplore.ieee.org/document/10229173/,3,Research Article,IEEE,"Acoustic wave equation modeling,numerical solv...",Solving the wave equation is an essential step...,Date of Publication: 24 August 2023,IEEE Geoscience and Remote Sensing Letters,"Jingbo Zou,Cai Liu,Chao Song,Pengfei Zhao,Jing...","(Department of Geophysics, College of Geo-Expl...","China,China,China,China,China,China,China,China",1,2,2,8
3,Simulating Multicomponent Elastic Seismic Wave...,https://ieeexplore.ieee.org/document/10054624/,7,Research Article,IEEE,"Deep learning,elastic media,frequency domain,p...",Simulating seismic wave propagation by solving...,Date of Publication: 28 February 2023,IEEE Geoscience and Remote Sensing Letters,"Chao Song,Yang Liu,Pengfei Zhao,Tianshuo Zhao,...",(State Key Laboratory of Shale Oil and Gas Enr...,"China,China,China,China,China,China,China,Chin...",1,3,3,12
4,Simulating Multicomponent Elastic Seismic Wave...,https://ieeexplore.ieee.org/document/10054624/,7,Research Article,IEEE,"Deep learning,elastic media,frequency domain,p...",Simulating seismic wave propagation by solving...,Date of Publication: 28 February 2023,IEEE Geoscience and Remote Sensing Letters,"Chao Song,Yang Liu,Pengfei Zhao,Tianshuo Zhao,...",(State Key Laboratory of Shale Oil and Gas Enr...,"China,China,China,China,China,China,China,Chin...",1,3,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Seismic Inversion Based on Acoustic Wave Equat...,https://ieeexplore.ieee.org/document/10017252/,13,Research Article,IEEE,"Acoustic wave equations,data normalization,phy...",Seismic inversion is a significant tool for ex...,Date of Publication: 16 January 2023,IEEE Transactions on Geoscience and Remote Sen...,"Yijie Zhang,Xueyu Zhu,Jinghuai Gao",(School of Information and Communications Engi...,"China,China",1,1,1,3
72,Physics-Informed Neural Networks for Solving U...,https://ieeexplore.ieee.org/document/10723708/,0,Conference Paper,IEEE,"underwater acoustic field,physics-informed neu...",This paper proposes a fast computational metho...,25 October 2024,2024 OES China Ocean Acoustics (COA),"Yuxiang Gao,Peng Xiao,Zhenglin Li,Yuxiang Gao,...","(School of Ocean Engineering and Technology, S...","China,China,China,China,China,China",1,0,0,6
73,Seismic Inversion Based on Acoustic Wave Equat...,https://ieeexplore.ieee.org/document/10017252/,13,Research Article,IEEE,"Acoustic wave equations,data normalization,phy...",Seismic inversion is a significant tool for ex...,Date of Publication: 16 January 2023,IEEE Transactions on Geoscience and Remote Sen...,"Yijie Zhang,Xueyu Zhu,Jinghuai Gao",(School of Information and Communications Engi...,"China,China",1,1,1,3
74,Physics-Informed Neural Networks for Solving U...,https://ieeexplore.ieee.org/docume